In [3]:
### Perform NEP analysis with squidpy
import anndata
import pandas as pd
import squidpy as sq
import matplotlib.pyplot as plt
import matplotlib as mpl
import time, os, sys
import glob
import warnings
import numpy as np
import seaborn as sns
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# import myocardial infarction dataset
path_to_csv = './../../../../../../MI_heart_paper/data/cell_table_final.csv'
obs = pd.read_csv(path_to_csv)
output_zscore = './../../../../../github/Comparison/20250218_results_MI/squidpy_zscore_knn5_delaunay_t.csv'
output_count = './../../../../../github/Comparison/20250218_results_MI/squidpy_count_knn5_delaunay_t.csv'

In [6]:
#filter out exclude cell types
ignore_cell_types = ['exclude']
obs = obs[~obs['final_cell_type'].isin(ignore_cell_types)]

# add marker files to it, as SpatialLDA needs them
obs['D'] = np.random.randint(1, 101, size=len(obs))
obs['E'] = np.random.randint(1, 101, size=len(obs))

# Ensure obs has a string-based index
obs.index = obs.index.astype(str)  

# Load dataframe into anndata object
X = obs[['D', 'E']]
X = X.values
adata = anndata.AnnData(X)
adata.obs = obs
adata

AnnData object with n_obs × n_vars = 563180 × 2
    obs: 'fov', 'label', 'cell_size', 'X_centroid', 'Y_centroid', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'cell_meta_cluster', 'timepoint', 'region', 'region_name', 'refined_cell_type', 'final_cell_type', 'endocardial_annotation', 'exclude_annotation', 'artefact', 'distance_from_lumen', 'lumen_bin', 'size_filter', 'D', 'E'

In [7]:
# get spatial coordinates to be in obsm
adata.obsm['spatial'] = obs[['X_centroid', 'Y_centroid']].values
# make "refined_cell_types" a categorical variable
adata.obs['final_cell_type'] = pd.Categorical(adata.obs['final_cell_type'])

In [10]:
# Run NEP per 'fov'
fov_list = adata.obs['fov'].unique()

# Dictionary to store fov-specific AnnData objects
fov_adata_dict = {}

# Loop over each fov
for fov in fov_list:
    # Create a subset of adata for the specific FOV
    fov_adata = adata[adata.obs['fov'] == fov].copy()

    # Perform spatial neighbors analysis on the subset
    sq.gr.spatial_neighbors(fov_adata, coord_type = 'generic', delaunay=False, n_neighs=5)
    sq.gr.nhood_enrichment(fov_adata, cluster_key="final_cell_type", n_perms=300, seed=fov_list.tolist().index(fov), show_progress_bar=False)
    # Store the result in the dictionary
    fov_adata_dict[fov] = fov_adata


/Users/chiaraschiller/miniconda3/envs/squidpy_env/lib/python3.10/site-packages/squidpy/gr/_nhood.py:202: RuntimeWarning: invalid value encountered in divide
  zscore = (count - perms.mean(axis=0)) / perms.std(axis=0)


In [11]:
# extract and save both, the z-score and the interaction count NEP matrix

def flatten_nep_matrices(fov_adata_dict, nep_mode='zscore', output_path=None):
    """
    Flattens NEP matrices (z-score or count) from a dictionary of FOV AnnData objects and returns a pivoted DataFrame.
    
    Parameters:
        fov_adata_dict (dict): Dictionary of FOV AnnData objects.
        nep_mode (str): Either 'zscore' or 'count' to select the enrichment type.
        output_path (str): Path to save the output CSV file.
    
    Returns:
        pd.DataFrame: Pivoted DataFrame containing the flattened NEP values.
    """
    assert nep_mode in ['zscore', 'count'], "nep_mode must be either 'zscore' or 'count'"
    
    flattened_dfs = []
    
    for fov, fov_adata in fov_adata_dict.items():
        # Extract the relevant NEP matrix
        nep_matrix = fov_adata.uns['final_cell_type_nhood_enrichment'][nep_mode]
        cell_types = fov_adata.obs['final_cell_type'].cat.categories
        
        assert len(cell_types) == nep_matrix.shape[0] == nep_matrix.shape[1], "Dimension mismatch!"
        
        nep_df = pd.DataFrame(nep_matrix, index=cell_types, columns=cell_types)
        flattened_df = nep_df.stack().reset_index()
        flattened_df.columns = ['rowname', 'colname', nep_mode]
        flattened_df['combined'] = flattened_df['rowname'] + '_' + flattened_df['colname']
        flattened_df['fov'] = fov
        
        flattened_dfs.append(flattened_df[['fov', 'combined', nep_mode]])
    
    big_dataframe = pd.concat(flattened_dfs, ignore_index=True)
    final_dataframe = big_dataframe.pivot(index='fov', columns='combined', values=nep_mode)
    final_dataframe.reset_index(inplace=True)
    
    # Store dataframe
    final_dataframe.to_csv(output_path, index=False)
    
    return final_dataframe


# Run for counts and z-scores
final_dataframe = flatten_nep_matrices(fov_adata_dict, 
                                          nep_mode='count', 
                                          output_path=output_count
                                          )
final_dataframe = flatten_nep_matrices(fov_adata_dict, 
                                          nep_mode='zscore', 
                                          output_path=output_zscore
                                          )
print(final_dataframe)

combined         fov  Cardiomyocytes Ankrd1+_Cardiomyocytes  \
0             24h_83                            -187.018141   
1             24h_86                            -185.926283   
2             48h_76                            -127.464162   
3             48h_79                            -140.657357   
4              4h_96                            -149.908091   
5              4h_97                            -106.424775   
6         Control_12                             -21.677321   
7         Control_13                              -4.497080   
8         Control_14                              -6.450905   

combined  Cardiomyocytes Ankrd1+_Cardiomyocytes Ankrd1+  \
0                                            215.567120   
1                                            251.608116   
2                                            136.819765   
3                                            188.037582   
4                                            180.484976   
5              